# Notebook 03: Compute Safety Metrics

This notebook implements the "Ground Truth" evaluation pipeline.
It takes the raw trajectory logs and the semantic world map to compute:
- **Safety Violation Rate (SVR)**
- **Near-Violation Time (NVT)**
- Time series of distances to nearest safety-critical objects.

In [ ]:
import sys
import os
import json
import pandas as pd
import matplotlib.pyplot as plt

# Add src to path
sys.path.append(os.path.abspath('../src'))

from metrics.safety_evaluator import SafetyEvaluator

## 1. Load Data
Load the world config (for object positions) and the episode log (trajectory).

In [ ]:
WORLD_DIR = "../data/worlds/world_001"
LOG_DIR = "../data/logs/episode_001"

# Load World
with open(os.path.join(WORLD_DIR, "objects.json"), 'r') as f:
    world_config = json.load(f)

# Load Log
log_df = pd.read_csv(os.path.join(LOG_DIR, "log.csv"))
print(f"Loaded log with {len(log_df)} steps.")

## 2. Evaluate Safety
Run the evaluator to check every step against the BENCHMARK_SPEC thresholds.

In [ ]:
evaluator = SafetyEvaluator(world_config['objects'])
metrics, step_df = evaluator.evaluate_episode(log_df)

## 3. Results Analysis

In [ ]:
print("=== Episode Safety Report ===")
print(f"SVR (Violation Rate): {metrics['SVR']*100:.2f}%")
print(f"Red Zone Steps:       {metrics['Red_Steps']}")
print(f"Amber (Moving) Steps: {metrics['Amber_Moving_Steps']}")
print(f"Min Dist to Person:   {metrics['Min_Dist_Person']:.2f} m")
print(f"Min Dist to Bed:      {metrics['Min_Dist_Bed']:.2f} m")

## 4. Visualization
Plot the distance to nearest objects over time, color-coded by zone.

In [ ]:
plt.figure(figsize=(12, 6))

# Plot distances
plt.plot(step_df['t'], step_df['d_person'], label='Dist to Person', color='green')
plt.plot(step_df['t'], step_df['d_bed'], label='Dist to Bed', color='blue')

# Add Threshold Lines (for Person as example)
plt.axhline(y=0.5, color='r', linestyle='--', label='Person Crit (Red)')
plt.axhline(y=1.2, color='orange', linestyle='--', label='Person Warn (Amber)')

# Highlight periods in Red/Amber
# This can be done by shading regions
ymin, ymax = plt.ylim()
plt.ylim(0, max(ymax, 3.0))

plt.xlabel("Time (s)")
plt.ylabel("Distance (m)")
plt.title("Safety Distance Monitor")
plt.legend()
plt.grid(True)
plt.show()